# Multimodal Single-Cell Integration: Creating a Sparse Matrix Dataset

This notebook goal is to generate a more efficient version of the dataset of the competition "Open Problems: Multimodal Single-Cell Integration", using scipy's sparse matrices.

Using sparse matrices will lead to:
- smaller file sizes
- faster loading
- much smaller memory footprint (so that you can actually keep all data in memory)

(Especially for the multiome data, which is very big and very sparse; not so much for the citeseq data which is smaller and only ~75% sparse).

The downside is that we cannot use the nice pandas DataFrame anymore. Instead, each "*xxx.h5*" file is converted into two files:
- One "*xxx_values.sparse*" file that can be loaded with `scipy.sparse.load_npz` and contains all the values of the corresponding dataframe (i.e. the result of `df.values` in a sparse format)
- One "*xxx_idxcol.npz*" file that can be loaded with `np.load` and contains the values of the index and the columns of the corresponding dataframe (i.e the results of `df.index` and `df.columns`)

For convenience, the csv files are also converted into a more efficient parquet version.

The generated dataset is available [here](https://www.kaggle.com/datasets/fabiencrom/multimodal-single-cell-as-sparse-matrix).

In [1]:
!conda install pytables -y

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pytables


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.1                   |         openblas           1 KB  conda-forge
    blosc-1.21.1               |       h83bc5f7_3          47 KB  conda-forge
    certifi-2022.6.15          |     pyhd8ed1ab_1         154 KB  conda-forge
    hdf5-1.12.2  

In [2]:
import pandas as pd
import numpy as np
import scipy.sparse

# Conversion Functions

In [3]:
def convert_to_parquet(filename, out_filename):
    df = pd.read_csv(filename)
    df.to_parquet(out_filename + ".parquet")

In [4]:
import scipy
def convert_h5_to_sparse_csr(filename, out_filename, chunksize=2500):
    start = 0
    total_rows = 0

    sparse_chunks_data_list = []
    chunks_index_list = []
    columns_name = None
    while True:
        df_chunk = pd.read_hdf(filename, start=start, stop=start+chunksize)
        if len(df_chunk) == 0:
            break
        chunk_data_as_sparse = scipy.sparse.csr_matrix(df_chunk.to_numpy())
        sparse_chunks_data_list.append(chunk_data_as_sparse)
        chunks_index_list.append(df_chunk.index.to_numpy())

        if columns_name is None:
            columns_name = df_chunk.columns.to_numpy()
        else:
            assert np.all(columns_name == df_chunk.columns.to_numpy())

        total_rows += len(df_chunk)
        print(total_rows)
        if len(df_chunk) < chunksize: 
            del df_chunk
            break
        del df_chunk
        start += chunksize
        
    all_data_sparse = scipy.sparse.vstack(sparse_chunks_data_list)
    del sparse_chunks_data_list
    
    all_indices = np.hstack(chunks_index_list)
    
    scipy.sparse.save_npz(out_filename+"_values.sparse", all_data_sparse)
    np.savez(out_filename+"_idxcol.npz", index=all_indices, columns =columns_name)
    
    


# H5 -> Sparse Conversion

In [5]:
convert_h5_to_sparse_csr("../input/open-problems-multimodal/train_multi_targets.h5", "train_multi_targets")

2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
72500
75000
77500
80000
82500
85000
87500
90000
92500
95000
97500
100000
102500
105000
105942


In [6]:
convert_h5_to_sparse_csr("../input/open-problems-multimodal/train_multi_inputs.h5", "train_multi_inputs")

2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
72500
75000
77500
80000
82500
85000
87500
90000
92500
95000
97500
100000
102500
105000
105942


In [7]:
convert_h5_to_sparse_csr("../input/open-problems-multimodal/train_cite_targets.h5", "train_cite_targets")

2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
70988


In [8]:
convert_h5_to_sparse_csr("../input/open-problems-multimodal/train_cite_inputs.h5", "train_cite_inputs")

2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
70988


In [9]:
convert_h5_to_sparse_csr("../input/open-problems-multimodal/test_multi_inputs.h5", "test_multi_inputs")

2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
55935


In [10]:
convert_h5_to_sparse_csr("../input/open-problems-multimodal/test_cite_inputs.h5", "test_cite_inputs")

2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
48663


# CSV -> PARQUET conversion
For convenience, let us also convert the other files from CSV to the more efficient parquet format.

(Then just replace pd.read_csv(xxx.csv) by pd.read_parquet(xxx.parquet) to read the file into a pandas DataFrame)

In [11]:
convert_to_parquet("../input/open-problems-multimodal/metadata.csv", "metadata")

In [12]:
convert_to_parquet("../input/open-problems-multimodal/evaluation_ids.csv", "evaluation")

In [13]:
convert_to_parquet("../input/open-problems-multimodal/sample_submission.csv", "sample_submission")

In [14]:
!ls -lh

total 7.1G
---------- 1 root root  75K Aug 31 09:31 __notebook__.ipynb
-rw-r--r-- 1 root root 359M Aug 31 09:31 evaluation.parquet
-rw-r--r-- 1 root root 3.8M Aug 31 09:29 metadata.parquet
-rw-r--r-- 1 root root 252M Aug 31 09:31 sample_submission.parquet
-rw-r--r-- 1 root root 1.8M Aug 31 09:29 test_cite_inputs_idxcol.npz
-rw-r--r-- 1 root root 488M Aug 31 09:29 test_cite_inputs_values.sparse.npz
-rw-r--r-- 1 root root 8.4M Aug 31 09:26 test_multi_inputs_idxcol.npz
-rw-r--r-- 1 root root 1.7G Aug 31 09:26 test_multi_inputs_values.sparse.npz
-rw-r--r-- 1 root root 2.2M Aug 31 09:12 train_cite_inputs_idxcol.npz
-rw-r--r-- 1 root root 712M Aug 31 09:12 train_cite_inputs_values.sparse.npz
-rw-r--r-- 1 root root 1.5M Aug 31 09:06 train_cite_targets_idxcol.npz
-rw-r--r-- 1 root root  36M Aug 31 09:06 train_cite_targets_values.sparse.npz
-rw-r--r-- 1 root root 9.5M Aug 31 09:06 train_multi_inputs_idxcol.npz
-rw-r--r-- 1 root root 2.8G Aug 31 09:06 train_multi_inputs_values.sparse.npz
-rw-r--